# Create 5-degree monthly average gridded dataset for SIF and XCO2
---

In [1]:
import sys
sys.path.insert(0, "../src")

import xarray
import data_utils as utils

In [ ]:
DS_sif = xarray.open_dataset("../data/exp_pro/OCO2_Lite_SIF740.nc")
DS_xco2 = xarray.open_dataset("../data/exp_pro/OCO2_Lite_XCO2_land.nc")

In [ ]:
df_sif_monthly = (
    utils.regrid(DS_sif, res=5)
    .groupby(["lon", "lat"])
    .resample("1MS")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)
df_xco2_monthly = (
    utils.regrid(DS_xco2, res=5)
    .groupby(["lon", "lat"])
    .resample("1MS")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)

# Merge dataframes
DS_grid = (
    pd.merge(df_sif_monthly, df_xco2_monthly, on=["lon", "lat", "time"], how="outer")
    .set_index(["lon", "lat", "time"])
    .to_xarray()
)
DS_grid

In [ ]:
DS_grid.to_netcdf("../data/exp_pro/OCO2_5deg_monthly.nc", format="NETCDF4")
DS_grid.close()